## Here is a simple visual tool for understanding the dependency of the two candidates and the pronoun. 
### I replaced all the correct candidate as 'Blake' and all the wrong candidate as 'Elliott'. They are both unisex names. The correct candidate is marked as red, while the wrong one is marked as green. The pronoun is marked as blue. 
### At the bottom I sorted some rules I think are useful. 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
from termcolor import colored

print(os.listdir("../input/gap-coreference/repository"))

# Any results you write to the current directory are saved as output.

In [ ]:
DATA_ROOT = '../input/'
GAP_DATA_FOLDER = os.path.join(DATA_ROOT, 'gap-coreference/repository/google-research-datasets-gap-coreference-83135f2/')
SUB_DATA_FOLDER = os.path.join(DATA_ROOT, 'gendered-pronoun-resolution')
FAST_TEXT_DATA_FOLDER = os.path.join(DATA_ROOT, 'fasttext-crawl-300d-2m')

# Import Data

In [ ]:
test_df_path = os.path.join(GAP_DATA_FOLDER, 'gap-development.tsv')
train_df_path = os.path.join(GAP_DATA_FOLDER, 'gap-test.tsv')
dev_df_path = os.path.join(GAP_DATA_FOLDER, 'gap-validation.tsv')

train_df = pd.read_csv(train_df_path, sep='\t')
test_df = pd.read_csv(test_df_path, sep='\t')
dev_df = pd.read_csv(dev_df_path, sep='\t')

#pd.options.display.max_colwidth = 1000

In [ ]:
def PrepData(data):
    newdata = data.copy()
    newdata.Text 
    #the order matters
    newdata.Text = newdata[['Text', 'Pronoun', 'Pronoun-offset']].apply(lambda x : x[0][:x[2]] + 'z'*len(x[1]) +x[0][(x[2] + len(x[1]) ):]  , axis = 1) 
    
    newdata.Text = newdata[['Text', 'A', 'A-coref']].apply(lambda x : x[0].replace(x[1], ' Blake ') if x[2] == True else x[0].replace(x[1], ' Elliott '), axis = 1) 
    newdata.Text = newdata[['Text', 'B', 'B-coref']].apply(lambda x : x[0].replace(x[1], ' Blake ') if x[2] == True else x[0].replace(x[1], ' Elliott ') , axis = 1) 
    
    return newdata

test = PrepData(test_df) 
train = PrepData(train_df)
dev  = PrepData(dev_df)

In [ ]:
from spacy.lang.en import English
from spacy.pipeline import DependencyParser
import spacy
from nltk import Tree
nlp = spacy.load('en_core_web_sm')
spacy.explain("VBN")
spacy.explain('ADP')

In [ ]:
import spacy
from spacy import displacy

nlp = spacy.load('en_core_web_sm')
#doc = nlp(u"Autonomous cars shift insurance liability toward manufacturers")
#displacy.render(doc, style='dep', jupyter=True)

In [ ]:

class Node:
    def __init__(self, name, lefts, rights,  pos_, tag_, dep_, parent=None , side = 'left'):
        self.name = name
        self.parent = parent
        self.lefts = lefts
        self.rights = rights
        self.side = side
        self.pos_ = pos_
        self.tag_ = tag_
        self.dep_ = dep_ 
        self.text = str(self.name)

def print_tree(current_node, pronoun, left_childattr='lefts', right_childattr = 'rights',  nameattr='name', indent='', last='updown' ):
    if hasattr(current_node, nameattr):
        name = lambda node: getattr(node, nameattr)
    else:
        name = lambda node: str(node)
        
    """ Creation of balanced lists for "up" branch and "down" branch. """
    up = [x for x in current_node.lefts if x.pos_ !='SPACE']
    down = [x for x in current_node.rights if x.pos_ !='SPACE']
    """ Printing of "up" branch. """
    for child in up:     
        next_last = 'up' if up.index(child) is 0 else ''
        next_indent = '{0}{1}{2}'.format(indent, ' ' if 'up' in last else '│', ' ' * (len(current_node.text + current_node.tag_ ) + 1))
        print_tree(child, pronoun, left_childattr, right_childattr, nameattr, next_indent, next_last)

    """ Printing of current node. """
    if last == 'up': start_shape = '┌'
    elif last == 'down': start_shape = '└'
    elif last == 'updown': start_shape = ' '
    else: start_shape = '├'

    if up: end_shape = '┤'
    elif down: end_shape = '┐'
    else: end_shape = ''
    if current_node.text =='Blake':
        print('{0}{1}{2}{3}'.format(indent, start_shape, colored(current_node.text +'&'+current_node.tag_, 'red'), end_shape))
    elif current_node.text =='Elliott':
        print('{0}{1}{2}{3}'.format(indent, start_shape, colored(current_node.text +'&'+ current_node.tag_, 'green'), end_shape))
        
    elif current_node.text  in ['zz','zzz','zzzz']:
        print('{0}{1}{2}{3}'.format(indent, start_shape, colored(pronoun, 'blue'), end_shape))
    else:
        print('{0}{1}{2}{3}'.format(indent, start_shape, current_node.text +'&' + current_node.tag_, end_shape))

    """ Printing of "down" branch. """
    for child in down:
        next_last = 'down' if down.index(child) is len(down) - 1 else ''
        next_indent = '{0}{1}{2}'.format(indent, ' ' if 'down' in last else '│', ' ' * (len(current_node.text + current_node.tag_ ) + 1))
        print_tree(child, pronoun, left_childattr,right_childattr, nameattr, next_indent, next_last)

In [ ]:
length = []
for i in range(10):
    print('This is', i)
     
    doc = nlp(dev.iloc[i].Text)
    
    for sent in doc.sents:
        print(sent)
        print_tree(Node(sent.root, list(sent.root.lefts), list(sent.root.rights), sent.root.pos_,sent.root.tag_,sent.root.dep_), dev.iloc[i].Pronoun )
    

In [ ]:
from spacy.lang.en import English
from spacy.pipeline import DependencyParser
import spacy
from nltk import Tree

## Type of Sentences 
### 1. A, B and pronoun appears in the same sentence. 
#### 1.1 one is in the cluase, and the other is the subjective to do something to the pronoun. 
#### 1.2. Pronoun appears at the first, 
### 2. There are sentences with A, B at first only.
#### 2.1 The pronoun appears as the main character in following sentence. 


 ##  Rules
 #### 1). If the second sentence starts with a verb which is token.tag_ =VBD, then it means the first sentence's subject is still the subject of the verb, the '.' can be replaced with ',' . If it is VBN, then the subject of previous sentence is current object. 
 #### 2). If the second sentence starts with the pronoun and no any entity, it should be the main person previous sentences were talking aobut. 
 #### 3). When the pronoun is in the the time Adverbial Clause with ``when'', then the first name followed is the entity. 
 #### 4). Sb verb  (sth) with/in he/her sth. then he/her here is the Sb. 
 #### 5). if the pronoun appear in the last sentence, normally it is the first entity mentioned in the first sentence. 
 #### 6). if the pronoun and both the two choices appear in the same sentence, the other sentneces can be ignored. 